In [1]:
import os
import cv2
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import time
import joblib
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
DATADIR = '/content/drive/MyDrive/flowers'
IMG_SIZE = 180
CATEGORIES = ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']

In [4]:
from tqdm import tqdm
data = []
def create_data():
  for category in CATEGORIES:
    path = os.path.join(DATADIR, category)
    class_num = CATEGORIES.index(category)
    for img in tqdm(os.listdir(path)):
      try:
        img_array = cv2.imread(os.path.join(path, img))
        new_array = cv2.resize(img_array, (IMG_SIZE, IMG_SIZE))
        data.append([new_array,class_num])
      except Exception as e:
        pass


In [5]:
create_data()
print(len(data))

100%|██████████| 984/984 [00:06<00:00, 144.97it/s]

4323


In [6]:
X = []
Y = []

for features, labels in tqdm(data):
  X.append(features)
  Y.append(labels)

100%|██████████| 4323/4323 [00:00<00:00, 1525105.24it/s]


In [7]:
# num_classes = Y['labels'].nunique()
# print("Total number of classes:", num_classes)

In [8]:
type(X)

list

In [9]:
x = np.array(X)
y = np.array(Y)

In [10]:
np.shape(X)

(4323, 180, 180, 3)

In [11]:
x_train, x_test, y_train, y_test=train_test_split(x, y, test_size=0.3, random_state=10)

In [12]:
x_train.shape

(3026, 180, 180, 3)

**Applying CNN**

In [13]:
#CNN

model = Sequential()
model.add(Conv2D(filters=64, kernel_size = (3,3), padding = 'Same', activation='relu', input_shape=(IMG_SIZE, IMG_SIZE, 3)))
model.add(Conv2D(filters=64, kernel_size = (3,3), padding = 'Same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(filters=128, kernel_size = (3,3), padding = 'Same', activation='relu'))
model.add(Conv2D(filters=128, kernel_size = (3,3), padding = 'Same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(filters=256, kernel_size = (3,3), padding = 'Same', activation='relu'))
model.add(Conv2D(filters=256, kernel_size = (3,3), padding = 'Same', activation='relu'))
model.add(Conv2D(filters=256, kernel_size = (3,3), padding = 'Same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(filters=512, kernel_size = (3,3), padding = 'Same', activation='relu'))
model.add(Conv2D(filters=512, kernel_size = (3,3), padding = 'Same', activation='relu'))
model.add(Conv2D(filters=512, kernel_size = (3,3), padding = 'Same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Conv2D(filters=512, kernel_size = (3,3), padding = 'Same', activation='relu'))
model.add(Conv2D(filters=512, kernel_size = (3,3), padding = 'Same', activation='relu'))
model.add(Conv2D(filters=512, kernel_size = (3,3), padding = 'Same', activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))

model.add(Flatten())

#ANN

model.add(Dense(4096, activation='relu'))
model.add(Dense(4096, activation='relu'))
model.add(Dense(5, activation='softmax'))



In [14]:
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [15]:
# S=time.time()
# model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))
# E=time.time()

In [16]:
# test_loss, test_acc = model.evaluate(x_test, y_test)
# #joblib.dump(cnnmodel, 'cnn_model.joblib')
# print(f"Test accuracy: {test_acc}")

In [17]:
# model.summary()

|       Models       | Train Acc | Test Acc |
|:------------------:|:---------:|:--------:|
|         ANN        |   43.23   |   30.45  |
| ANN  (PCA(n=1000)) |   99.04   |   47.26  |
|  ANN  (PCA(n=300)) |   86.32   |   41.25  |
|         CNN        |   80.60   |   69.08  |

In [18]:
datagen=ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2,

)


In [19]:
History = model.fit(
    datagen.flow(x_train, y_train, batch_size=128),
    epochs = 20, validation_data = (x_test, y_test),
    steps_per_epoch = x_train.shape[0]//128

)

/usr/local/lib/python3.10/dist-packages/keras/preprocessing/image.py:1861: UserWarning: This ImageDataGenerator specifies `featurewise_center`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/preprocessing/image.py:1871: UserWarning: This ImageDataGenerator specifies `featurewise_std_normalization`, but it hasn't been fit on any training data. Fit it first by calling `.fit(numpy_data)`.
  warnings.warn(


Epoch 1/20
23/23 [==============================] - 102s 3s/step - loss: 1.5856 - accuracy: 0.2943 - val_loss: 1.5517 - val_accuracy: 0.3531
Epoch 2/20
23/23 [==============================] - 39s 2s/step - loss: 1.5454 - accuracy: 0.3499 - val_loss: 1.5091 - val_accuracy: 0.3832
Epoch 3/20
23/23 [==============================] - 37s 2s/step - loss: 1.4877 - accuracy: 0.3730 - val_loss: 1.4524 - val_accuracy: 0.3616
Epoch 4/20
23/23 [==============================] - 36s 2s/step - loss: 1.4384 - accuracy: 0.3892 - val_loss: 1.3673 - val_accuracy: 0.4171
Epoch 5/20
23/23 [==============================] - 37s 2s/step - loss: 1.4295 - accuracy: 0.3651 - val_loss: 1.3501 - val_accuracy: 0.4402
Epoch 6/20
23/23 [==============================] - 38s 2s/step - loss: 1.3851 - accuracy: 0.3954 - val_loss: 1.2737 - val_accuracy: 0.4873
Epoch 7/20
23/23 [==============================] - 38s 2s/step - loss: 1.3818 - accuracy: 0.4027 - val_loss: 1.3070 - val_accuracy: 0.4248
Epoch 8/20
23/23 [=

In [20]:
x_train.shape

(3026, 180, 180, 3)